# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

## Import Library

In [135]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

## Import Model

In [136]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant, Chapter

## Data extraction from model

In [137]:
class LaboratoryData:

    def __init__(self, lab = Laboratory):
        self._lab = lab
        self._laboratories = lab.objects.all()

    @property
    def laboratories(self): 
        return self._laboratories.values()
    
    def get_assistant(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        assistants = laboratory.assistants.all()
        return assistants.values()
    
    def get_module(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        modules = laboratory.modules.all()
        return modules.values()
    
class ModuleData:

    def __init__(self, module = Module):
        self._module = module
        self._modules = module.objects.all()

    @property
    def modules(self): 
        return self._modules.values()
    
    def get_module(self,id):
        module = self._modules.filter(id=id).first()
        return module
    
    def get_module_chapter(self, id):
        module = self.get_module(id)
        return module.chapters.all().values()
    
    def get_group(self, id):
        module = self.get_module(id)
        return module.groups.all().values()
    
    def get_laboratory(self, id):
        module = self.get_module(id)
        return module.laboratory
    
        
class ChapterData:

    def __init__(self, chapter = Chapter):
        self._chapter = chapter
        self._chapters = chapter.objects.all()

    @property
    def chapters(self): 
        return self._chapters.values()
    
    def get_chapter(self,id):
        chapter = self._chapters.filter(id=id).first()
        return chapter
    
    def get_chapter_module(self, id):
        chapter = self.get_chapter(id)
        return chapter.module

class ParticipantData:

    def __init__(self, participant = Participant):
        self._participant = participant
        self._participants = participant.objects.all()
    
    @property
    def participants(self):
        return self._participants.values()
    
    def group(self, id):
        group_memberships = self._participants.filter(id=id).first().group_memberships.all()
        groups = []
        for group_membership in group_memberships:
            group = group_membership.group
            groups.append(group)
        return groups
    
class GroupData:
    def __init__(self, group = Group):
        self._group = group
        self._groups = group.objects.all()

    @property
    def groups(self):
        return self._groups.values()
    
    def get_group(self, id):
        group = self._groups.filter(id=id).first()
        return group
    
    def get_module_group(self, module_id):
        groups = self._groups.filter(module_id=module_id)
        return groups.values()
    
    def participants(self, id):
        group = self._groups.filter(id=id).first()
        group_memberships = group.group_memberships.all()
        participants = []
        for group_membership in group_memberships:
            participant = group_membership.participant
            participants.append(participant)
        return participants
    
    def get_participant_schedule(self, id):
        participants = self.participants(id)
        participant_schedule = []
        for participant in participants:
            participant_schedule.append(participant.regular_schedule)
        return participant_schedule
    
    def get_group_schedule(self, id):
        participant_schedule = self.get_participant_schedule(id)
        if len(participant_schedule) == 0:
            return {}
        days = participant_schedule[0].keys()
        merged_schedule = {day: {} for day in days}
        for day in days:
            for time_slot in participant_schedule[0][day]:
                is_available = all(schedule[day][time_slot] for schedule in participant_schedule)
                merged_schedule[day][time_slot] = is_available
        return merged_schedule
                
            
class AssistantData:

    def __init__(self, assistant = Assistant):
        self._assistant = assistant
        self._assistants = assistant.objects.all()

    @property
    def assistants(self):
        return self._assistants.values()

    
    def get_assistant(self, id):
        assistant = self._assistants.filter(id=id).first()
        return assistant
    
    def get_assistant_modules(self,id):
        assistant = self._assistant.filter(id=id).first()
        assistant_memberships = assistant.assistant_memberships.all()
        modules = []
        for assistant_membership in assistant_memberships:
            module = assistant_membership.module
            modules.append(module)
        return modules


## Genetic Algorithm

In [138]:
import pandas as pd

In [139]:
# Assigning data to variables
lab_data = LaboratoryData()
module_data = ModuleData()
participant_data = ParticipantData()
group_data = GroupData()
assistant_data = AssistantData()
chapter_data = ChapterData()

In [140]:
group_data.get_module_group(1).count()

36

In [141]:
pd.DataFrame(participant_data.participants)

,id,name,nim,semester_id,regular_schedule
0,5,Setio Ningrum,3332210016,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
1,6,Ruth Anandina,3332210051,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
2,7,ADHITAMA WIRA YUDHA,3332190029,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
3,8,FARHAN NUGROHO,3332190072,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
4,9,BAGAS NURYANTO,3332190080,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
...,...,...,...,...,...
102,55,Maulana Ali Akbar,3332210082,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
103,65,Naga Tunggal,3332210004,1,"{'Friday': {'Shift1': True, 'Shift2': False, '..."
104,86,Randy eleanor,3332210057,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
105,106,MAHESA NURUL VIKAR,3332200087,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."


In [142]:
module_data.get_group(1)

<QuerySet [{'id': 1, 'name': 'PL-1', 'module_id': 1}, {'id': 2, 'name': 'PL-2', 'module_id': 1}, {'id': 3, 'name': 'PL-3', 'module_id': 1}, {'id': 4, 'name': 'PL-4', 'module_id': 1}, {'id': 5, 'name': 'PL-5', 'module_id': 1}, {'id': 6, 'name': 'PL-6', 'module_id': 1}, {'id': 7, 'name': 'PL-7', 'module_id': 1}, {'id': 8, 'name': 'PL-8', 'module_id': 1}, {'id': 9, 'name': 'PL-9', 'module_id': 1}, {'id': 10, 'name': 'PL-10', 'module_id': 1}, {'id': 11, 'name': 'PL-11', 'module_id': 1}, {'id': 12, 'name': 'PL-12', 'module_id': 1}, {'id': 13, 'name': 'PL-13', 'module_id': 1}, {'id': 14, 'name': 'PL-14', 'module_id': 1}, {'id': 15, 'name': 'PL-15', 'module_id': 1}, {'id': 16, 'name': 'PL-16', 'module_id': 1}, {'id': 17, 'name': 'PL-17', 'module_id': 1}, {'id': 18, 'name': 'PL-18', 'module_id': 1}, {'id': 19, 'name': 'PL-19', 'module_id': 1}, {'id': 20, 'name': 'PL-20', 'module_id': 1}, '...(remaining elements truncated)...']>

### Gene Representation for Scheduling Problem

In [143]:
from collections import namedtuple
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

In [144]:
# Gene Representation
# 1. Laboratory
# 2. Module
# 3. Module chapter
# 4. Group
# 5. Assistant
# 6. Time Slot : Date, Day, Time
# 7. Availability: json, e.g. {"Friday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Monday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Tuesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Saturday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": false, "Shift6": true}, "Thursday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": true}, "Wednesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": false, "Shift5": false, "Shift6": true}}

class Gene:
    def __init__(self, laboratory, module, module_chapter, group, assistant, time_slot: TimeSlot):
        self.laboratory = laboratory
        self.module = module
        self.module_chapter = module_chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot
        
    def __repr__(self):
        return f"Gene(lab={self.laboratory}, module={self.module}, module_chapter={self.module_chapter}, group={self.group}, assistant={self.assistant}, time_slot={self.time_slot})"
    
    def generate(self):
        return {
            "lab": self.laboratory,
            "module": self.module,
            "module_chapter": self.module_chapter,
            "group": self.group,
            "assistant": self.assistant,
            "time_slot": self.time_slot
        }
    

In [145]:
# Test Gene
gene = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[0]["id"],assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift1"))
gene2 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[1]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift2"))
gene3 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[2]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Monday", "Shift1"))

In [146]:
gene.time_slot == gene3.time_slot

False

In [147]:
group_data.groups

<QuerySet [{'id': 1, 'name': 'PL-1', 'module_id': 1}, {'id': 2, 'name': 'PL-2', 'module_id': 1}, {'id': 3, 'name': 'PL-3', 'module_id': 1}, {'id': 4, 'name': 'PL-4', 'module_id': 1}, {'id': 5, 'name': 'PL-5', 'module_id': 1}, {'id': 6, 'name': 'PL-6', 'module_id': 1}, {'id': 7, 'name': 'PL-7', 'module_id': 1}, {'id': 8, 'name': 'PL-8', 'module_id': 1}, {'id': 9, 'name': 'PL-9', 'module_id': 1}, {'id': 10, 'name': 'PL-10', 'module_id': 1}, {'id': 11, 'name': 'PL-11', 'module_id': 1}, {'id': 12, 'name': 'PL-12', 'module_id': 1}, {'id': 13, 'name': 'PL-13', 'module_id': 1}, {'id': 14, 'name': 'PL-14', 'module_id': 1}, {'id': 15, 'name': 'PL-15', 'module_id': 1}, {'id': 16, 'name': 'PL-16', 'module_id': 1}, {'id': 17, 'name': 'PL-17', 'module_id': 1}, {'id': 18, 'name': 'PL-18', 'module_id': 1}, {'id': 19, 'name': 'PL-19', 'module_id': 1}, {'id': 20, 'name': 'PL-20', 'module_id': 1}, '...(remaining elements truncated)...']>

#### Gene-Level Constraints: Hard Constraints

In [148]:
# Constraint Class for Constraint Function
# 1. Ensure that each chapter is assigned to the correct module.
# 2. Ensure that each module is assigned to the correct lab.
# 3. Ensure that each group is assigned to the correct module.
# 4. Ensure that each assistant is assigned to the correct lab.

class ConstraintChecker:
    def __init__(self, gene, lab_data: LaboratoryData, module_data: ModuleData, group_data: GroupData, assistant_data: AssistantData, chapter_data: ChapterData):
        self.gene = gene
        self.lab_data = lab_data
        self.module_data = module_data
        self.group_data = group_data
        self.assistant_data = assistant_data
        self.chapter_data = chapter_data

    def chapter_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        chapter = self.chapter_data.get_chapter(self.gene.module_chapter)
        return module == chapter.module
    
    def module_lab_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        return module.laboratory.id == self.gene.laboratory
    
    def group_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        group = self.group_data.get_group(self.gene.group)
        return module == group.module
    
    def assistant_lab_constraint(self):
        assistant = self.assistant_data.get_assistant(self.gene.assistant)
        return assistant.laboratory.id == self.gene.laboratory
    
    def check(self):
        return self.chapter_module_constraint() and self.module_lab_constraint() and self.group_module_constraint() and self.assistant_lab_constraint()

In [149]:
assistant_data.get_assistant(1).laboratory.id

1

### Chromosome Representation for Scheduling Problem

In [150]:
# Chromosome.py
# Chromosome Representation
# 1. Gene

class Chromosome:
    def __init__(self, genes):
        self.genes = genes
        
    def __repr__(self):
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def __eq__(self, other):
        return self.genes == other.genes and self.fitness == other.fitness
    
    def __hash__(self):
        return hash((self.genes, self.fitness))
    
    def __str__(self) -> str:
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def generate(self):
        return Chromosome(self.genes, self.fitness)
    

In [151]:
# test chromosome
chromosome = Chromosome([gene, gene2, gene3])

#### Chromosome-Level Constraints

#### Objective Function, 
satisfying hard and soft constraints.
1. Hard Constraints
    - Each course must be assigned to a single room in a single time slot.
    - Each room can only accommodate a specified course at a time.
    - Each lecturer can only teach one course at a time.
    - Each lecturer can only assist one group of students at a time.
    - Each student can only take one course at a time.
    - Each course must be assigned to a single time slot.
    - Each course must be assigned to a single lecturer.
    - Each course must be assigned up to a specified student group.
    - Each course must be finished within a specified time slot.
    - Course held for 5 days a week.
    - No two groups can be assigned to the same lab and time slot simultaneously.
    - Make sure each group is assigned to all the courses they need. In this case, each group must be assigned to each chapter of the course module.

2. Soft Constraints
    - Maximum units that can be held simultaneously on a module the same is 2 modules per week.
    - The period between each unit is at least 2 days.
    - The number of groups for each course in 1 shift is 3 groups

Objective class
1. Minimize conflict: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.
2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

In [152]:
# Objective Class for Fitness Function
# 1. Minimize Conflicts: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

class MinimizeConflicts:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.conflicts = namedtuple("Conflicts", ["time_slot", "lab"])
        self.conflicts_time_slot = set()
        self.assigned_time_slot = set()
        self.conflict_count = 0


    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Conflicts Time Slot: {self.conflicts_time_slot}, Assigned Time Slot: {self.assigned_time_slot}, Conflict Count: {self.conflict_count}"
    
    def __eq__(self, other):
        return self.chromosome == other.chromosome and self.conflicts_time_slot == other.conflicts_time_slot and self.assigned_time_slot == other.assigned_time_slot and self.conflict_count == other.conflict_count
    
    def calculate_conflicts(self):
        for gene in self.chromosome.genes:
            if (gene.laboratory, gene.time_slot) in self.assigned_time_slot:
                self.conflicts_time_slot.add(self.conflicts(gene.time_slot, gene.laboratory))
                self.conflict_count += 1
            else:
                self.assigned_time_slot.add((gene.laboratory, gene.time_slot))
        return self.conflict_count

    def calculate_fitness(self):
        self.calculate_conflicts()
        return self.conflict_count


In [153]:
a = MinimizeConflicts(chromosome)
a.calculate_fitness()

0

In [154]:
# 2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.

from collections import defaultdict
from itertools import combinations

class MaximizeResourceUtilization:
    def __init__(self, chromosome:list, max_groups_per_assistant, max_shift_per_assistant):
        self.chromosome = chromosome
        self.max_groups_per_assistant = max_groups_per_assistant
        self.max_shift_per_assistant = max_shift_per_assistant
        self.assistants_assignments = defaultdict(list)
        self.assistants_shifts = defaultdict(list)
        self.overload_penalty = 0

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Max Groups Per Assistant: {self.max_groups_per_assistant}, Max Shift Per Assistant: {self.max_shift_per_assistant}, Assistants Assignments: {self.assistants_assignments}, Assistants Shifts: {self.assistants_shifts}, Overload Penalty: {self.overload_penalty}"
    
    def calculate_assistants_assignments(self):
        self.assistants_assignments.clear()
        for gene in self.chromosome.genes:
            self.assistants_assignments[gene.assistant].append(gene.group)
        return self.assistants_assignments
    
    def calculate_assistants_shifts(self):
        self.assistants_shifts.clear()
        for gene in self.chromosome.genes:
            self.assistants_shifts[gene.assistant].append(gene.time_slot)
        return self.assistants_shifts
    
    def calculate_overload_penalty(self):
        self.overload_penalty = 0
        self.calculate_assistants_assignments()
        self.calculate_assistants_shifts()
        for assistant , assignments in self.assistants_assignments.items():
            assignments_penalty = max(0, len(assignments) - self.max_groups_per_assistant)
            shift_penalty = (max(0, len(self.assistants_shifts[assistant]) - self.max_shift_per_assistant))
            self.overload_penalty += assignments_penalty + shift_penalty
        return self.overload_penalty
    
    def calculate_fitness(self):
        self.calculate_overload_penalty()
        return self.overload_penalty
    
b = MaximizeResourceUtilization(chromosome, 2, 2)
b.calculate_fitness()

        

2

In [155]:
group_data.get_group_schedule(gene.group)

{'Friday': {'Shift1': False,
  'Shift2': True,
  'Shift3': False,
  'Shift4': True,
  'Shift5': False,
  'Shift6': True},
 'Monday': {'Shift1': True,
  'Shift2': True,
  'Shift3': False,
  'Shift4': False,
  'Shift5': True,
  'Shift6': False},
 'Tuesday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Saturday': {'Shift1': False,
  'Shift2': False,
  'Shift3': True,
  'Shift4': False,
  'Shift5': True,
  'Shift6': True},
 'Thursday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': True,
  'Shift6': False},
 'Wednesday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': True,
  'Shift5': True,
  'Shift6': True}}

In [156]:
group_data.get_group_schedule(gene.group)["Friday"]["Shift1"]

False

In [157]:
# 3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

class ParticipantAvailability:
    def __init__(self, chromosome, group_data: GroupData):
        self.chromosome = chromosome
        self.group_data = group_data
        self.unsatified_participants = defaultdict(list)
        self.unsatified_participants_count = 0

    def calculate_unsatisfied_participants(self):
        self.unsatified_participants.clear()
        for gene in self.chromosome.genes:
            participant_availability = group_data.get_group_schedule(gene.group)
            if not participant_availability[gene.time_slot.day][gene.time_slot.shift]:
                self.unsatified_participants[gene.group].append(gene.time_slot)
                self.unsatified_participants_count += 1
        return self.unsatified_participants_count
    
    def calculate_fitness(self):
        self.calculate_unsatisfied_participants()
        return self.unsatified_participants_count
    
c = ParticipantAvailability(chromosome, group_data)
c.calculate_fitness()
            

2

In [158]:
c.unsatified_participants

defaultdict(list,
            {1: [TimeSlot(date='2021-01-01', day='Friday', shift='Shift1')],
             3: [TimeSlot(date='2021-01-01', day='Monday', shift='Shift1')]})

In [159]:
# 4. Maximize Weekly Chapter Coverage: Maximize the number of chapters covered in a week by ensuring that each chapter is assigned to a time slot that is available to them.

In [160]:
module_data.get_module_chapter(gene.module_chapter)

<QuerySet [{'id': 1, 'name': 'U-1', 'module_id': 1}, {'id': 2, 'name': 'U-2', 'module_id': 1}, {'id': 3, 'name': 'U-3', 'module_id': 1}, {'id': 4, 'name': 'U-4', 'module_id': 1}, {'id': 5, 'name': 'U-5', 'module_id': 1}, {'id': 6, 'name': 'U-6', 'module_id': 1}, {'id': 7, 'name': 'U-7', 'module_id': 1}, {'id': 8, 'name': 'U-8', 'module_id': 1}]>

In [161]:
chromosome.genes

[Gene(lab=1, module=1, module_chapter=1, group=1, assistant=1, time_slot=TimeSlot(date='2021-01-01', day='Friday', shift='Shift1')),
 Gene(lab=1, module=1, module_chapter=1, group=2, assistant=1, time_slot=TimeSlot(date='2021-01-01', day='Friday', shift='Shift2')),
 Gene(lab=1, module=1, module_chapter=1, group=3, assistant=1, time_slot=TimeSlot(date='2021-01-01', day='Monday', shift='Shift1'))]

#### Constants: 
1. Number of days in a week
2. Number of time slots in a day

In [162]:
# Constant.py
class Constant:
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shifts = ["Shift1", "Shift2", "Shift3", "Shift4", "Shift5", "Shift6"]

### Population Initialization

In [163]:
# Population Representation
# 1. Chromosome
# 2. Population Size
# 3. Mutation Rate
# 4. Crossover Rate
# 5. Selection Strategy
# 6. Fitness Function

from random import random, randint, choice
from copy import deepcopy
from math import ceil
import numpy as np

class Population:
    def __init__(self, chromosomes, fitness_function):
        self.chromosomes = chromosomes
        self.fitnesess = [fitness_function(chromosome) for chromosome in self.chromosomes]

    def __repr__(self):
        return f"Chromosomes: {self.chromosomes}"
    
    def __eq__(self, other):
        return self.chromosomes == other.chromosomes
    
    def __hash__(self):
        return hash(self.chromosomes)
    

### Algorithm

##### Strategy

Selection Strategy
1. Roulette Wheel Selection
2. Tournament Selection
3. Rank Selection

In [164]:
class SelectionStrategy:
    @staticmethod
    def roulette_wheel(population, fitness_function):
        fitnesses = [fitness_function(chromosome) for chromosome in population.chromosomes]
        total_fitness = sum(fitnesses)
        probabilities = [fitness / total_fitness for fitness in fitnesses]
        return np.random.choice(population.chromosomes, p=probabilities)
    
    @staticmethod
    def tournament(population, fitness_function, **kwargs):
        tournament_size = kwargs.get("tournament_size", 2)
        tournament_candidates = np.random.choice(population.chromosomes, tournament_size)
        return max(tournament_candidates, key=fitness_function)
    
    def random(self, population):
        return choice(population.chromosomes)
    
    @staticmethod
    def elitism(population, fitness_value, **kwargs):
        elitism_size = kwargs.get("elitism_size", 1)
        return Population(population.chromosomes[:elitism_size], fitness_value)
    
    @staticmethod
    def select(population, fitness_function, selection_size, selection_strategy):
        return Population([selection_strategy(population, fitness_function) for _ in range(selection_size)])


##### Operators

In [165]:

from datetime import datetime, timedelta

In [166]:
group_count = group_data.groups.count()
chapter_count = chapter_data.chapters.count()
group_count * chapter_count

288

In [167]:
# Population Generator: Generate Gene, Generate Chromosome, Generate Population
# 1. Generate Gene
# 2. Generate Chromosome
# 3. Generate Population

class PopulationGenerator:
    def __init__(self, lab_data: LaboratoryData, module_data: ModuleData, participant_data: ParticipantData, group_data: GroupData, assistant_data: AssistantData, chapter_data: ChapterData):
        self.lab_data = lab_data
        self.module_data = module_data
        self.participant_data = participant_data
        self.group_data = group_data
        self.assistant_data = assistant_data
        self.chapter_data = chapter_data
        
    def generate_gene(self):
        while True:
            # Generate gene
            laboratory = choice(self.lab_data.laboratories)["id"]
            module = choice(self.lab_data.get_module(laboratory))["id"]
            module_chapter = choice(self.module_data.get_module_chapter(module))["id"]
            group = choice(self.module_data.get_group(module))["id"]
            assistant = choice(self.lab_data.get_assistant(laboratory))["id"]

            # Generate time slot
            module_start_date = self.module_data.get_module(module).start_date
            module_end_date = self.module_data.get_module(module).end_date
            module_duration = (module_end_date - module_start_date).days + 1
            module_weeks = ceil(module_duration / 7)
            
            week = randint(0, module_weeks-1)
            day = choice(Constant.days)
            shift = choice(Constant.shifts)
            date = module_start_date + timedelta(days=week*7 + Constant.days.index(day))
            time_slot = TimeSlot(date, day, shift)
            gene = Gene(laboratory, module, module_chapter, group, assistant, time_slot)
            constraint_checker = ConstraintChecker(gene, self.lab_data, self.module_data, self.group_data, self.assistant_data, self.chapter_data)
            if constraint_checker.check():
                return gene
    
    def generate_chromosome(self, **kwargs):
        chromosome_size = kwargs.get("chromosome_size", self.calculate_chromosome_size())
        genes = []
        for _ in range(chromosome_size):
            genes.append(self.generate_gene())
        return Chromosome(genes)
    
    def calculate_chromosome_size(self):
        chromosome_size = 0
        for module in self.module_data.modules:
            module_chapter_count = self.module_data.get_module_chapter(module["id"]).count()
            group_count = self.module_data.get_group(module["id"]).count()
            chromosome_size += module_chapter_count * group_count
        return chromosome_size
        
    def generate_population(self, population_size = 10, fitness_function = None):
        chromosomes = []
        for _ in range(population_size):
            chromosomes.append(self.generate_chromosome())
        return Population(chromosomes, fitness_function)

In [168]:
# GeneticOperators.py

from datetime import timedelta

class GeneticOperators:
    @staticmethod
    def generate_genes():
        while True:
            # Generate gene
            laboratory = choice(lab_data.laboratories)["id"]
            module = choice(lab_data.get_module(laboratory))["id"]
            module_chapter = choice(module_data.get_module_chapter(module))["id"]
            group = choice(module_data.get_group(module))["id"]
            assistant = choice(lab_data.get_assistant(laboratory))["id"]

            # Generate time slot
            time_slot = GeneticOperators.generate_timeslot(module)
            gene = Gene(laboratory, module, module_chapter, group, assistant, time_slot)
            constraint_checker = ConstraintChecker(gene, lab_data, module_data, group_data, assistant_data, chapter_data)
            if constraint_checker.check():
                return gene
            
    def generate_timeslot(module_id):
        while True:
            # Generate time slot
            module_start_date = module_data.get_module(module_id).start_date
            module_end_date = module_data.get_module(module_id).end_date
            module_duration = (module_end_date - module_start_date).days + 1
            module_weeks = ceil(module_duration / 7)
            
            week = randint(0, module_weeks-1)
            day = choice(Constant.days)
            shift = choice(Constant.shifts)
            date = module_start_date + timedelta(days=week*7 + Constant.days.index(day))
            time_slot = TimeSlot(date, day, shift)
            return time_slot

    @staticmethod
    def crossover(parent1, parent2, num_crossover_points, crossover_probability):
        if random() > crossover_probability:
            return parent1, parent2
        crossover_points = sorted([randint(0, len(parent1.genes)) for _ in range(num_crossover_points)])
        parent1_genes = deepcopy(parent1.genes)
        parent2_genes = deepcopy(parent2.genes)
        for i in range(0, len(crossover_points), 2):
            start = crossover_points[i]
            end = crossover_points[i+1] if i+1 < len(crossover_points) else len(parent1.genes)
            for j in range(start, end):
                parent1_genes[j], parent2_genes[j] = parent2_genes[j], parent1_genes[j]
        return Chromosome(parent1_genes, 0), Chromosome(parent2_genes, 0)
    
    @staticmethod
    def mutate(parent, mutation_size, mutation_probability):
        if random() > mutation_probability:
            return parent
        mutated_genes = deepcopy(parent.genes)
        for _ in range(mutation_size):
            mutated_genes[randint(0, len(mutated_genes)-1)] = GeneticOperators.generate_genes()
        return Chromosome(mutated_genes, 0)
    
    @staticmethod
    def select(population, fitness_function, selection_size, selection_strategy):
        return selection_strategy.select(population, selection_size, fitness_function)
    
    @staticmethod
    def repair(chromosome, fitness_function):
        for gene in chromosome.genes:
            if not group_data.get_group_schedule(gene.group)[gene.time_slot.day][gene.time_slot.shift]:
                gene.time_slot = GeneticOperators.generate_timeslot(gene.module)
        chromosome.fitness = fitness_function(chromosome)
        return chromosome
    
    @staticmethod
    def fitness_function(chromosome):
        chromosome.fitness = MinimizeConflicts(chromosome).calculate_fitness() + MaximizeResourceUtilization(chromosome, 8, 16).calculate_fitness() + ParticipantAvailability(chromosome, group_data).calculate_fitness()
        return chromosome.fitness


In [169]:
# GeneticAlgorithm.py
class GeneticAlgorithm:
    def __init__(self, population_generator: PopulationGenerator, fitness_function, selection_strategy: SelectionStrategy, crossover_probability, mutation_probability, num_crossover_points, mutation_size, elitism_size):
        self.population_generator = population_generator
        self.fitness_function = fitness_function
        self.selection_strategy = selection_strategy
        self.crossover_probability = crossover_probability
        self.mutation_probability = mutation_probability
        self.num_crossover_points = num_crossover_points
        self.mutation_size = mutation_size
        self.elitism_size = elitism_size

    def __repr__(self):
        return f"Population Generator: {self.population_generator}, Fitness Function: {self.fitness_function}, Selection Strategy: {self.selection_strategy}, Crossover Probability: {self.crossover_probability}, Mutation Probability: {self.mutation_probability}, Number of Crossover Points: {self.num_crossover_points}, Mutation Size: {self.mutation_size}, Elitism Size: {self.elitism_size}"
    
    def __eq__(self, other):
        return self.population_generator == other.population_generator and self.fitness_function == other.fitness_function and self.selection_strategy == other.selection_strategy and self.crossover_probability == other.crossover_probability and self.mutation_probability == other.mutation_probability and self.num_crossover_points == other.num_crossover_points and self.mutation_size == other.mutation_size and self.elitism_size == other.elitism_size
    
    def __hash__(self):
        return hash((self.population_generator, self.fitness_function, self.selection_strategy, self.crossover_probability, self.mutation_probability, self.num_crossover_points, self.mutation_size, self.elitism_size))
    
    def __str__(self) -> str:
        return f"Population Generator: {self.population_generator}, Fitness Function: {self.fitness_function}, Selection Strategy: {self.selection_strategy}, Crossover Probability: {self.crossover_probability}, Mutation Probability: {self.mutation_probability}, Number of Crossover Points: {self.num_crossover_points}, Mutation Size: {self.mutation_size}, Elitism Size: {self.elitism_size}"
    
    def evolve(self, population):
        new_population = Population([], self.fitness_function)
        elitism = SelectionStrategy.elitism(population, self.elitism_size)
        new_population.chromosomes.extend(elitism.chromosomes)
        while len(new_population.chromosomes) < len(population.chromosomes):
            parent1 = SelectionStrategy.select(population, self.fitness_function, 1, self.selection_strategy).chromosomes[0]
            parent2 = SelectionStrategy.select(population, self.fitness_function, 1, self.selection_strategy).chromosomes[0]
            child1, child2 = GeneticOperators.crossover(parent1, parent2, self.num_crossover_points, self.crossover_probability)
            child1 = GeneticOperators.mutate(child1, self.mutation_size, self.mutation_probability)
            child2 = GeneticOperators.mutate(child2, self.mutation_size, self.mutation_probability)
            child1 = GeneticOperators.repair(child1, self.fitness_function)
            child2 = GeneticOperators.repair(child2, self.fitness_function)
            new_population.chromosomes.extend([child1, child2])
        return new_population

    def run(self, max_generations, population_size, verbose=False):
        population = self.population_generator.generate_population(population_size, self.fitness_function)
        for generation in range(max_generations):
            population = self.evolve(population)
            if verbose:
                print(f"Generation {generation}: {population.chromosomes[0].fitness}")
        return population

In [170]:
#GeneticAlgorithmBuilder.py
class GeneticAlgorithmBuilder:
    def __init__(self):
        self.parameters = {
            "population_size": 10,
            "mutation_rate": 0.1,
            "crossover_rate": 0.8,
            "num_crossover_points": 2,
            "mutation_size": 1,
            "selection_size": 2,
            "elitism_size": 1,
            "max_generations": 10,
            "max_groups_per_assistant": 2,
            "max_shift_per_assistant": 2,
            "tournament_size": 2,
            "selection_strategy": SelectionStrategy().roulette_wheel,
            "fitness_function": GeneticAlgorithmBuilder.fitness_function
        }

    def set_population_size(self, population_size):
        self.parameters["population_size"] = population_size
        return self
    
    def set_mutation_rate(self, mutation_rate):
        self.parameters["mutation_rate"] = mutation_rate
        return self
    
    def set_crossover_rate(self, crossover_rate):
        self.parameters["crossover_rate"] = crossover_rate
        return self
    
    def set_num_crossover_points(self, num_crossover_points):
        self.parameters["num_crossover_points"] = num_crossover_points
        return self
    
    def set_mutation_size(self, mutation_size):
        self.parameters["mutation_size"] = mutation_size
        return self
    
    def set_selection_size(self, selection_size):
        self.parameters["selection_size"] = selection_size
        return self
    
    def set_elitism_size(self, elitism_size):
        self.parameters["elitism_size"] = elitism_size
        return self
    
    def set_max_generations(self, max_generations):
        self.parameters["max_generations"] = max_generations
        return self
    
    def set_max_groups_per_assistant(self, max_groups_per_assistant):
        self.parameters["max_groups_per_assistant"] = max_groups_per_assistant
        return self
    
    def set_max_shift_per_assistant(self, max_shift_per_assistant):
        self.parameters["max_shift_per_assistant"] = max_shift_per_assistant
        return self
    
    def set_tournament_size(self, tournament_size):
        self.parameters["tournament_size"] = tournament_size
        return self
    
    def set_selection_strategy(self, selection_strategy):
        self.parameters["selection_strategy"] = selection_strategy
        return self
    
    def set_fitness_function(self, fitness_function):
        self.parameters["fitness_function"] = fitness_function
        return self
    
    def build(self):
        return GeneticAlgorithm(**self.parameters)

In [171]:
#Test
population_generator = PopulationGenerator(lab_data, module_data, participant_data, group_data, assistant_data, chapter_data)
fitness_function = GeneticOperators.fitness_function
selection_strategy = SelectionStrategy.tournament
crossover_probability = 0.8
mutation_probability = 0.5
num_crossover_points = 5
mutation_size = 2
elitism_size = 1
max_generations = 10
max_groups_per_assistant = 6
max_shift_per_assistant = 16
tournament_size = 2
genetic_algorithm = GeneticAlgorithm(population_generator, fitness_function, selection_strategy, crossover_probability, mutation_probability, num_crossover_points, mutation_size, elitism_size)

In [172]:
test = genetic_algorithm.run(max_generations, 3, verbose=True)

TypeError: Population.__init__() missing 1 required positional argument: 'fitness_function'

In [148]:
GeneticOperators.fitness_function(test.chromosomes[0])


827